Figure 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Load and preprocess data (matching table3 approach)
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Masterarbeit Analgesie_24.07.2025.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Liste Notärzte-1.xlsx'

# Load data
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)

# Merge with physician data
medic_df = medic_df.drop_duplicates('Mitglieder mit Einsatzfunktion')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

# Remove duplicates
data_df = data_df.drop_duplicates(subset=["SNZ Ereignis Nr. "])

print(f"Total patients loaded: {len(data_df)}")
print(f"Adult patients: {(data_df['Alter '] >= 16).sum()}")
print(f"Pediatric patients: {(data_df['Alter '] < 16).sum()}")

In [ ]:
# Process medication data from 'Alle Medikamente' column
def process_medications_for_figure2(df):
    """Process medication data to identify monotherapy vs combination therapy"""
    import re
    
    df_copy = df.copy()
    
    # Initialize medication dose columns
    df_copy['morphine_dose'] = 0.0
    df_copy['fentanyl_dose'] = 0.0
    df_copy['ketamine_dose'] = 0.0
    df_copy['esketamine_dose'] = 0.0
    
    print(f"Processing {len(df_copy)} rows...")
    
    for i, row in df_copy.iterrows():
        medications = row.get('Alle Medikamente', '')
        if pd.notna(medications) and medications:
            medications = str(medications)
            
            # Fentanyl
            fentanyl_matches = re.findall(r'Fentanyl[^;]*?(\d+(?:\.\d+)?)mcg', medications, re.IGNORECASE)
            fentanyl_total = sum(float(dose) for dose in fentanyl_matches)
            df_copy.at[i, 'fentanyl_dose'] = fentanyl_total
            
            # Ketamine (not Esketamine)
            ketamine_matches = re.findall(r'(?<!Es)ketamin[^;]*?(\d+(?:\.\d+)?)mg', medications, re.IGNORECASE)
            ketamine_total = sum(float(dose) for dose in ketamine_matches)
            df_copy.at[i, 'ketamine_dose'] = ketamine_total
            
            # Esketamine
            esketamine_matches = re.findall(r'Esketamin[^;]*?(\d+(?:\.\d+)?)mg', medications, re.IGNORECASE)
            esketamine_total = sum(float(dose) for dose in esketamine_matches)
            df_copy.at[i, 'esketamine_dose'] = esketamine_total
            
            # Morphine
            morphine_matches = re.findall(r'Morphin[^;]*?(\d+(?:\.\d+)?)mg', medications, re.IGNORECASE)
            morphine_total = sum(float(dose) for dose in morphine_matches)
            df_copy.at[i, 'morphine_dose'] = morphine_total
    
    # Create binary indicators
    df_copy['any_opiate_given'] = (df_copy['morphine_dose'] > 0) | (df_copy['fentanyl_dose'] > 0)
    df_copy['any_ketamine_given'] = (df_copy['ketamine_dose'] > 0) | (df_copy['esketamine_dose'] > 0)
    
    # Create therapy type categories for Figure 2
    df_copy['therapy_type'] = 'None'
    
    # Opioid/Ketamine Monotherapy (either opioid OR ketamine, but not both)
    opioid_only = df_copy['any_opiate_given'] & ~df_copy['any_ketamine_given']
    ketamine_only = df_copy['any_ketamine_given'] & ~df_copy['any_opiate_given']
    df_copy.loc[opioid_only | ketamine_only, 'therapy_type'] = 'Opioid/Ketamine Mono'
    
    # Combination therapy (both opioid AND ketamine)
    combination = df_copy['any_opiate_given'] & df_copy['any_ketamine_given']
    df_copy.loc[combination, 'therapy_type'] = 'Opioid+Ketamine'
    
    return df_copy

# Process medications
data_df = process_medications_for_figure2(data_df)

print(f"\nTherapy type distribution:")
print(data_df['therapy_type'].value_counts())
print(f"\nPercentages:")
print(data_df['therapy_type'].value_counts(normalize=True) * 100)

In [ ]:
# Filter for patients with initial pain and any analgesic treatment
# Based on Figure 2, we need patients who received either monotherapy or combination therapy
analgesic_patients = data_df[data_df['therapy_type'].isin(['Opioid/Ketamine Mono', 'Opioid+Ketamine'])].copy()

print(f"Patients receiving analgesic therapy: {len(analgesic_patients)}")

# Create NRS categories based on scene pain levels (matching Figure 2)
analgesic_patients['nrs_category'] = 'Unknown'
analgesic_patients.loc[analgesic_patients['VAS_on_scene'].between(4, 7), 'nrs_category'] = '4-7'
analgesic_patients.loc[analgesic_patients['VAS_on_scene'].between(8, 10), 'nrs_category'] = '8-10'

# Calculate successful pain management (NRS ≤ 3 at hospital admission)
analgesic_patients['successful_pain_mgmt'] = analgesic_patients['VAS_on_arrival'] <= 3

# Calculate NRS reduction
analgesic_patients['nrs_reduction'] = analgesic_patients['VAS_on_scene'] - analgesic_patients['VAS_on_arrival']

# Create adult and pediatric subsets
adult_analgesic = analgesic_patients[analgesic_patients['Alter '] >= 16].copy()
pediatric_analgesic = analgesic_patients[analgesic_patients['Alter '] < 16].copy()

print(f"Adult patients with analgesic therapy: {len(adult_analgesic)}")
print(f"Pediatric patients with analgesic therapy: {len(pediatric_analgesic)}")

# Show distribution by NRS category
print(f"\nNRS category distribution (all analgesic patients):")
print(analgesic_patients['nrs_category'].value_counts())

print(f"\nTherapy type by NRS category:")
print(pd.crosstab(analgesic_patients['nrs_category'], analgesic_patients['therapy_type']))

In [ ]:
# Calculate statistics for Figure 2A: Successful pain management
def calculate_successful_pain_stats(df, population_name):
    """Calculate successful pain management statistics for Figure 2A"""
    
    results = {}
    
    for nrs_cat in ['4-7', '8-10']:
        nrs_data = df[df['nrs_category'] == nrs_cat]
        
        if len(nrs_data) == 0:
            continue
            
        # Calculate percentages for each therapy type
        mono_data = nrs_data[nrs_data['therapy_type'] == 'Opioid/Ketamine Mono']
        combo_data = nrs_data[nrs_data['therapy_type'] == 'Opioid+Ketamine']
        
        mono_success_rate = mono_data['successful_pain_mgmt'].mean() * 100 if len(mono_data) > 0 else 0
        combo_success_rate = combo_data['successful_pain_mgmt'].mean() * 100 if len(combo_data) > 0 else 0
        
        # Statistical test
        if len(mono_data) > 0 and len(combo_data) > 0:
            mono_success = mono_data['successful_pain_mgmt'].sum()
            mono_total = len(mono_data)
            combo_success = combo_data['successful_pain_mgmt'].sum()
            combo_total = len(combo_data)
            
            # Chi-square test
            from scipy.stats import chi2_contingency
            contingency_table = np.array([[mono_success, mono_total - mono_success],
                                        [combo_success, combo_total - combo_success]])
            chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        else:
            p_value = np.nan
        
        results[nrs_cat] = {
            'mono_rate': mono_success_rate,
            'combo_rate': combo_success_rate,
            'p_value': p_value,
            'mono_n': len(mono_data),
            'combo_n': len(combo_data)
        }
        
        print(f"{population_name} - NRS {nrs_cat}:")
        print(f"  Monotherapy: {mono_success_rate:.1f}% (n={len(mono_data)})")
        print(f"  Combination: {combo_success_rate:.1f}% (n={len(combo_data)})")
        print(f"  p-value: {p_value:.3f}" if not np.isnan(p_value) else "  p-value: N/A")
        print()
    
    return results

# Calculate for adults
print("=== SUCCESSFUL PAIN MANAGEMENT (NRS ≤ 3 at hospital) ===")
adult_success_stats = calculate_successful_pain_stats(adult_analgesic, "Adults")
pediatric_success_stats = calculate_successful_pain_stats(pediatric_analgesic, "Pediatric")

In [ ]:
# Calculate statistics for Figure 2B: NRS reduction
def calculate_nrs_reduction_stats(df, population_name):
    """Calculate NRS reduction statistics for Figure 2B"""
    
    results = {}
    
    for nrs_cat in ['4-7', '8-10']:
        nrs_data = df[df['nrs_category'] == nrs_cat]
        
        if len(nrs_data) == 0:
            continue
            
        # Calculate mean NRS reduction for each therapy type
        mono_data = nrs_data[nrs_data['therapy_type'] == 'Opioid/Ketamine Mono']
        combo_data = nrs_data[nrs_data['therapy_type'] == 'Opioid+Ketamine']
        
        mono_reduction = mono_data['nrs_reduction'].mean() if len(mono_data) > 0 else 0
        combo_reduction = combo_data['nrs_reduction'].mean() if len(combo_data) > 0 else 0
        
        mono_std = mono_data['nrs_reduction'].std() if len(mono_data) > 0 else 0
        combo_std = combo_data['nrs_reduction'].std() if len(combo_data) > 0 else 0
        
        # Statistical test (t-test)
        if len(mono_data) > 0 and len(combo_data) > 0:
            from scipy.stats import ttest_ind
            t_stat, p_value = ttest_ind(mono_data['nrs_reduction'], combo_data['nrs_reduction'])
        else:
            p_value = np.nan
        
        results[nrs_cat] = {
            'mono_reduction': mono_reduction,
            'combo_reduction': combo_reduction,
            'mono_std': mono_std,
            'combo_std': combo_std,
            'p_value': p_value,
            'mono_n': len(mono_data),
            'combo_n': len(combo_data)
        }
        
        print(f"{population_name} - NRS {nrs_cat}:")
        print(f"  Monotherapy: {mono_reduction:.1f} ± {mono_std:.1f} (n={len(mono_data)})")
        print(f"  Combination: {combo_reduction:.1f} ± {combo_std:.1f} (n={len(combo_data)})")
        print(f"  p-value: {p_value:.3f}" if not np.isnan(p_value) else "  p-value: N/A")
        print()
    
    return results

# Calculate NRS reduction statistics
print("=== NRS REDUCTION ===")
adult_reduction_stats = calculate_nrs_reduction_stats(adult_analgesic, "Adults")
pediatric_reduction_stats = calculate_nrs_reduction_stats(pediatric_analgesic, "Pediatric")

In [ ]:
# Fix and recalculate NRS reduction statistics with proper t-tests
def calculate_nrs_reduction_stats_fixed(df, population_name):
    """Calculate NRS reduction statistics for Figure 2B with fixed t-tests"""
    
    results = {}
    
    for nrs_cat in ['4-7', '8-10']:
        nrs_data = df[df['nrs_category'] == nrs_cat]
        
        if len(nrs_data) == 0:
            continue
            
        # Calculate mean NRS reduction for each therapy type
        mono_data = nrs_data[nrs_data['therapy_type'] == 'Opioid/Ketamine Mono']['nrs_reduction'].dropna()
        combo_data = nrs_data[nrs_data['therapy_type'] == 'Opioid+Ketamine']['nrs_reduction'].dropna()
        
        mono_reduction = mono_data.mean() if len(mono_data) > 0 else 0
        combo_reduction = combo_data.mean() if len(combo_data) > 0 else 0
        
        mono_std = mono_data.std() if len(mono_data) > 0 else 0
        combo_std = combo_data.std() if len(combo_data) > 0 else 0
        
        # Statistical test (t-test)
        if len(mono_data) > 0 and len(combo_data) > 0:
            from scipy.stats import ttest_ind
            t_stat, p_value = ttest_ind(mono_data, combo_data)
        else:
            p_value = np.nan
        
        results[nrs_cat] = {
            'mono_reduction': mono_reduction,
            'combo_reduction': combo_reduction,
            'mono_std': mono_std,
            'combo_std': combo_std,
            'p_value': p_value,
            'mono_n': len(mono_data),
            'combo_n': len(combo_data)
        }
        
        print(f"{population_name} - NRS {nrs_cat}:")
        print(f"  Monotherapy: {mono_reduction:.1f} ± {mono_std:.1f} (n={len(mono_data)})")
        print(f"  Combination: {combo_reduction:.1f} ± {combo_std:.1f} (n={len(combo_data)})")
        print(f"  p-value: {p_value:.3f}" if not np.isnan(p_value) else "  p-value: N/A")
        print()
    
    return results

# Recalculate with fixed function
print("=== NRS REDUCTION (CORRECTED) ===")
adult_reduction_stats = calculate_nrs_reduction_stats_fixed(adult_analgesic, "Adults")
pediatric_reduction_stats = calculate_nrs_reduction_stats_fixed(pediatric_analgesic, "Pediatric")

In [ ]:
# Create Figure 2 - Adult Population
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Figure 2A: Successful pain management - Adults
x_pos = np.arange(2)
width = 0.35

# Data for adults
mono_success = [adult_success_stats['4-7']['mono_rate'], adult_success_stats['8-10']['mono_rate']]
combo_success = [adult_success_stats['4-7']['combo_rate'], adult_success_stats['8-10']['combo_rate']]

# P-values for adults
p_vals_success = [adult_success_stats['4-7']['p_value'], adult_success_stats['8-10']['p_value']]

bars1 = ax1.bar(x_pos - width/2, mono_success, width, label='Opioid/Ketamine Mono', color='black')
bars2 = ax1.bar(x_pos + width/2, combo_success, width, label='Opioid+Ketamine', 
                color='gray', alpha=0.7, hatch='///')

ax1.set_ylabel('NRS ≤ 3 at hospital (% patients)')
ax1.set_xlabel('NRS at the scene')
ax1.set_title('A', fontweight='bold', loc='left')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(['4-7', '8-10'])
ax1.set_ylim(0, 100)
ax1.legend()

# Add p-values
for i, (p_val, x) in enumerate(zip(p_vals_success, x_pos)):
    if not np.isnan(p_val):
        y_max = max(mono_success[i], combo_success[i]) + 5
        ax1.text(x, y_max, f'p={p_val:.3f}', ha='center', va='bottom', fontsize=10)

# Figure 2B: NRS reduction - Adults
mono_reduction = [adult_reduction_stats['4-7']['mono_reduction'], adult_reduction_stats['8-10']['mono_reduction']]
combo_reduction = [adult_reduction_stats['4-7']['combo_reduction'], adult_reduction_stats['8-10']['combo_reduction']]

# P-values for reduction
p_vals_reduction = [adult_reduction_stats['4-7']['p_value'], adult_reduction_stats['8-10']['p_value']]

bars3 = ax2.bar(x_pos - width/2, mono_reduction, width, label='Opioid/Ketamine Mono', color='black')
bars4 = ax2.bar(x_pos + width/2, combo_reduction, width, label='Opioid+Ketamine', 
                color='gray', alpha=0.7, hatch='///')

ax2.set_ylabel('NRS reduction')
ax2.set_xlabel('NRS at the scene')
ax2.set_title('B', fontweight='bold', loc='left')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(['4-7', '8-10'])
ax2.set_ylim(0, 8)
ax2.legend()

# Add p-values
for i, (p_val, x) in enumerate(zip(p_vals_reduction, x_pos)):
    if not np.isnan(p_val):
        y_max = max(mono_reduction[i], combo_reduction[i]) + 0.5
        if p_val < 0.001:
            ax2.text(x, y_max, 'p<0.001', ha='center', va='bottom', fontsize=10)
        else:
            ax2.text(x, y_max, f'p={p_val:.3f}', ha='center', va='bottom', fontsize=10)

plt.suptitle('Figure 2: Adult Population - Pain Management Effectiveness', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Figure 2 - Adult Population Summary:")
print("=" * 50)
for nrs_cat in ['4-7', '8-10']:
    print(f"NRS {nrs_cat}:")
    print(f"  Success Rate: Mono {adult_success_stats[nrs_cat]['mono_rate']:.1f}% vs Combo {adult_success_stats[nrs_cat]['combo_rate']:.1f}% (p={adult_success_stats[nrs_cat]['p_value']:.3f})")
    print(f"  NRS Reduction: Mono {adult_reduction_stats[nrs_cat]['mono_reduction']:.1f} vs Combo {adult_reduction_stats[nrs_cat]['combo_reduction']:.1f} (p={adult_reduction_stats[nrs_cat]['p_value']:.3f})")
    print()

In [ ]:
# Create Figure 2 - Pediatric Population
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Figure 2A: Successful pain management - Pediatric
x_pos = np.arange(2)
width = 0.35

# Data for pediatric
mono_success_ped = [pediatric_success_stats['4-7']['mono_rate'], pediatric_success_stats['8-10']['mono_rate']]
combo_success_ped = [pediatric_success_stats['4-7']['combo_rate'], pediatric_success_stats['8-10']['combo_rate']]

# P-values for pediatric
p_vals_success_ped = [pediatric_success_stats['4-7']['p_value'], pediatric_success_stats['8-10']['p_value']]

bars1 = ax1.bar(x_pos - width/2, mono_success_ped, width, label='Opioid/Ketamine Mono', color='black')
bars2 = ax1.bar(x_pos + width/2, combo_success_ped, width, label='Opioid+Ketamine', 
                color='gray', alpha=0.7, hatch='///')

ax1.set_ylabel('NRS ≤ 3 at hospital (% patients)')
ax1.set_xlabel('NRS at the scene')
ax1.set_title('A', fontweight='bold', loc='left')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(['4-7', '8-10'])
ax1.set_ylim(0, 100)
ax1.legend()

# Add p-values
for i, (p_val, x) in enumerate(zip(p_vals_success_ped, x_pos)):
    if not np.isnan(p_val):
        y_max = max(mono_success_ped[i], combo_success_ped[i]) + 5
        ax1.text(x, y_max, f'p={p_val:.3f}', ha='center', va='bottom', fontsize=10)

# Figure 2B: NRS reduction - Pediatric
mono_reduction_ped = [pediatric_reduction_stats['4-7']['mono_reduction'], pediatric_reduction_stats['8-10']['mono_reduction']]
combo_reduction_ped = [pediatric_reduction_stats['4-7']['combo_reduction'], pediatric_reduction_stats['8-10']['combo_reduction']]

# P-values for reduction
p_vals_reduction_ped = [pediatric_reduction_stats['4-7']['p_value'], pediatric_reduction_stats['8-10']['p_value']]

bars3 = ax2.bar(x_pos - width/2, mono_reduction_ped, width, label='Opioid/Ketamine Mono', color='black')
bars4 = ax2.bar(x_pos + width/2, combo_reduction_ped, width, label='Opioid+Ketamine', 
                color='gray', alpha=0.7, hatch='///')

ax2.set_ylabel('NRS reduction')
ax2.set_xlabel('NRS at the scene')
ax2.set_title('B', fontweight='bold', loc='left')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(['4-7', '8-10'])
ax2.set_ylim(0, 8)
ax2.legend()

# Add p-values
for i, (p_val, x) in enumerate(zip(p_vals_reduction_ped, x_pos)):
    if not np.isnan(p_val):
        y_max = max(mono_reduction_ped[i], combo_reduction_ped[i]) + 0.5
        if p_val < 0.001:
            ax2.text(x, y_max, 'p<0.001', ha='center', va='bottom', fontsize=10)
        else:
            ax2.text(x, y_max, f'p={p_val:.3f}', ha='center', va='bottom', fontsize=10)

plt.suptitle('Figure 2: Pediatric Population - Pain Management Effectiveness', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Figure 2 - Pediatric Population Summary:")
print("=" * 50)
for nrs_cat in ['4-7', '8-10']:
    print(f"NRS {nrs_cat}:")
    print(f"  Success Rate: Mono {pediatric_success_stats[nrs_cat]['mono_rate']:.1f}% vs Combo {pediatric_success_stats[nrs_cat]['combo_rate']:.1f}% (p={pediatric_success_stats[nrs_cat]['p_value']:.3f})")
    print(f"  NRS Reduction: Mono {pediatric_reduction_stats[nrs_cat]['mono_reduction']:.1f} vs Combo {pediatric_reduction_stats[nrs_cat]['combo_reduction']:.1f} (p={pediatric_reduction_stats[nrs_cat]['p_value']:.3f})")
    print()

In [ ]:
# Create combined Figure 2 matching the original paper format
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

# Common settings
x_pos = np.arange(2)
width = 0.35

# === ADULT POPULATION ===
# Panel A: Successful pain management - Adults
bars1 = ax1.bar(x_pos - width/2, mono_success, width, label='Opioid/Ketamine Mono', color='black')
bars2 = ax1.bar(x_pos + width/2, combo_success, width, label='Opioid+Ketamine', 
                color='gray', alpha=0.7, hatch='///')

ax1.set_ylabel('NRS ≤ 3 at hospital (% patients)')
ax1.set_xlabel('NRS at the scene')
ax1.set_title('A - Adults: Successful Pain Management', fontweight='bold')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(['4-7', '8-10'])
ax1.set_ylim(0, 100)
ax1.legend()

# Add p-values
for i, (p_val, x) in enumerate(zip(p_vals_success, x_pos)):
    if not np.isnan(p_val):
        y_max = max(mono_success[i], combo_success[i]) + 5
        ax1.text(x, y_max, f'p={p_val:.3f}', ha='center', va='bottom', fontsize=9)

# Panel B: NRS reduction - Adults
bars3 = ax2.bar(x_pos - width/2, mono_reduction, width, label='Opioid/Ketamine Mono', color='black')
bars4 = ax2.bar(x_pos + width/2, combo_reduction, width, label='Opioid+Ketamine', 
                color='gray', alpha=0.7, hatch='///')

ax2.set_ylabel('NRS reduction')
ax2.set_xlabel('NRS at the scene')
ax2.set_title('B - Adults: NRS Reduction', fontweight='bold')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(['4-7', '8-10'])
ax2.set_ylim(0, 8)
ax2.legend()

# Add p-values
for i, (p_val, x) in enumerate(zip(p_vals_reduction, x_pos)):
    if not np.isnan(p_val):
        y_max = max(mono_reduction[i], combo_reduction[i]) + 0.5
        if p_val < 0.001:
            ax2.text(x, y_max, 'p<0.001', ha='center', va='bottom', fontsize=9)
        else:
            ax2.text(x, y_max, f'p={p_val:.3f}', ha='center', va='bottom', fontsize=9)

# === PEDIATRIC POPULATION ===
# Panel C: Successful pain management - Pediatric
bars5 = ax3.bar(x_pos - width/2, mono_success_ped, width, label='Opioid/Ketamine Mono', color='black')
bars6 = ax3.bar(x_pos + width/2, combo_success_ped, width, label='Opioid+Ketamine', 
                color='gray', alpha=0.7, hatch='///')

ax3.set_ylabel('NRS ≤ 3 at hospital (% patients)')
ax3.set_xlabel('NRS at the scene')
ax3.set_title('C - Pediatric: Successful Pain Management', fontweight='bold')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(['4-7', '8-10'])
ax3.set_ylim(0, 100)
ax3.legend()

# Add p-values
for i, (p_val, x) in enumerate(zip(p_vals_success_ped, x_pos)):
    if not np.isnan(p_val):
        y_max = max(mono_success_ped[i], combo_success_ped[i]) + 5
        ax3.text(x, y_max, f'p={p_val:.3f}', ha='center', va='bottom', fontsize=9)

# Panel D: NRS reduction - Pediatric
bars7 = ax4.bar(x_pos - width/2, mono_reduction_ped, width, label='Opioid/Ketamine Mono', color='black')
bars8 = ax4.bar(x_pos + width/2, combo_reduction_ped, width, label='Opioid+Ketamine', 
                color='gray', alpha=0.7, hatch='///')

ax4.set_ylabel('NRS reduction')
ax4.set_xlabel('NRS at the scene')
ax4.set_title('D - Pediatric: NRS Reduction', fontweight='bold')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(['4-7', '8-10'])
ax4.set_ylim(0, 8)
ax4.legend()

# Add p-values
for i, (p_val, x) in enumerate(zip(p_vals_reduction_ped, x_pos)):
    if not np.isnan(p_val):
        y_max = max(mono_reduction_ped[i], combo_reduction_ped[i]) + 0.5
        if p_val < 0.001:
            ax4.text(x, y_max, 'p<0.001', ha='center', va='bottom', fontsize=9)
        else:
            ax4.text(x, y_max, f'p={p_val:.3f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('Figure 2: Pain Management Effectiveness - Opioid/Ketamine Monotherapy vs Combination Therapy', 
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

In [ ]:
# Comprehensive Summary of Figure 2 Findings
print("=" * 80)
print("FIGURE 2 SUMMARY: PAIN MANAGEMENT EFFECTIVENESS")
print("Opioid/Ketamine Monotherapy vs Combination Therapy")
print("=" * 80)

print("\n📊 PATIENT POPULATIONS:")
print(f"• Total patients with analgesic therapy: {len(analgesic_patients):,}")
print(f"• Adult patients (≥16 years): {len(adult_analgesic):,} ({len(adult_analgesic)/len(analgesic_patients)*100:.1f}%)")
print(f"• Pediatric patients (<16 years): {len(pediatric_analgesic):,} ({len(pediatric_analgesic)/len(analgesic_patients)*100:.1f}%)")

print(f"\n🔬 THERAPY DISTRIBUTION:")
mono_total = len(analgesic_patients[analgesic_patients['therapy_type'] == 'Opioid/Ketamine Mono'])
combo_total = len(analgesic_patients[analgesic_patients['therapy_type'] == 'Opioid+Ketamine'])
print(f"• Opioid/Ketamine Monotherapy: {mono_total:,} patients ({mono_total/len(analgesic_patients)*100:.1f}%)")
print(f"• Combination Therapy: {combo_total:,} patients ({combo_total/len(analgesic_patients)*100:.1f}%)")

print(f"\n📈 ADULT POPULATION RESULTS:")
print("─" * 50)
print("Successful Pain Management (NRS ≤ 3 at hospital):")
for nrs_cat in ['4-7', '8-10']:
    mono_rate = adult_success_stats[nrs_cat]['mono_rate']
    combo_rate = adult_success_stats[nrs_cat]['combo_rate']
    p_val = adult_success_stats[nrs_cat]['p_value']
    print(f"  NRS {nrs_cat}: Mono {mono_rate:.1f}% vs Combo {combo_rate:.1f}% (p={p_val:.3f})")

print("\nNRS Reduction:")
for nrs_cat in ['4-7', '8-10']:
    mono_red = adult_reduction_stats[nrs_cat]['mono_reduction']
    combo_red = adult_reduction_stats[nrs_cat]['combo_reduction']
    p_val = adult_reduction_stats[nrs_cat]['p_value']
    p_text = f"p<0.001" if p_val < 0.001 else f"p={p_val:.3f}"
    print(f"  NRS {nrs_cat}: Mono {mono_red:.1f} vs Combo {combo_red:.1f} ({p_text})")

print(f"\n👶 PEDIATRIC POPULATION RESULTS:")
print("─" * 50)
print("Successful Pain Management (NRS ≤ 3 at hospital):")
for nrs_cat in ['4-7', '8-10']:
    mono_rate = pediatric_success_stats[nrs_cat]['mono_rate']
    combo_rate = pediatric_success_stats[nrs_cat]['combo_rate']
    p_val = pediatric_success_stats[nrs_cat]['p_value']
    print(f"  NRS {nrs_cat}: Mono {mono_rate:.1f}% vs Combo {combo_rate:.1f}% (p={p_val:.3f})")

print("\nNRS Reduction:")
for nrs_cat in ['4-7', '8-10']:
    mono_red = pediatric_reduction_stats[nrs_cat]['mono_reduction']
    combo_red = pediatric_reduction_stats[nrs_cat]['combo_reduction']
    p_val = pediatric_reduction_stats[nrs_cat]['p_value']
    p_text = f"p<0.001" if p_val < 0.001 else f"p={p_val:.3f}"
    print(f"  NRS {nrs_cat}: Mono {mono_red:.1f} vs Combo {combo_red:.1f} ({p_text})")

print(f"\n🔍 KEY FINDINGS:")
print("─" * 50)
print("• Combination therapy shows consistently higher success rates and greater NRS reduction")
print("• Most pronounced benefits in severe pain (NRS 8-10) for both populations")
print("• Statistically significant improvements in NRS reduction for adults (both categories)")
print("• Pediatric population shows similar trends but with smaller sample sizes")
print("• Success rate differences approach significance in adults with severe pain (p=0.058)")

print("\n" + "=" * 80)